In [0]:
df_silver_municipio = spark.read.parquet("/mnt/dados/silver/municipio/tb_municipio.parquet")

In [0]:
df_silver_municipio.createOrReplaceTempView("tb_municipio_nova")

In [0]:
display(spark.sql("SELECT * FROM tb_municipio_nova"))

In [0]:
df_silver = spark.read.parquet("/mnt/dados/silver/dengue/tb_dengue.parquet")

In [0]:
df_silver.createOrReplaceTempView("tb_dengue_nova")

In [0]:
display(spark.sql("SELECT * FROM tb_dengue_nova"))

In [0]:
%sql
SELECT  
    c.ID_MUNICIPIO,
    m.MUNICIPIO,
    m.LATITUDE,
    m.LONGITUDE,
    m.ALTITUDE,
    c.DT_NOTIFICACAO,
    c.ANO,
    c.SEXO,
    c.POSSUI_FEBRE,
    c.POSSUI_NAUSEA,
    c.POSSUI_VOMITO,
    c.POSSUI_DOR_MUSCULAR,
    c.POSSUI_DOR_DE_CABECA,
    current_timestamp() AS DT_INGESTAO
FROM tb_dengue_nova c
INNER JOIN tb_municipio_nova m ON c.ID_MUNICIPIO = m.ID_MUNICIPIO

## TOTAL DE CASO DE DENGUE POR ANO

In [0]:
%sql
SELECT 
  ANO AS ANO,
  COUNT(*) AS TOTAL_CASOS
FROM tb_dengue_nova
GROUP BY ANO
ORDER BY ANO

## MUNICIPIO QUE TEVE MAIS CASOS DE DENGUE EM CADA ANO

In [0]:
%sql
SELECT 
  t.ANO,
  t.ID_MUNICIPIO,
  m.MUNICIPIO,
  t.CASOS_TOTAL
FROM (
  SELECT 
    ANO,
    ID_MUNICIPIO,
    COUNT(*) AS CASOS_TOTAL,
    ROW_NUMBER() OVER (PARTITION BY ANO ORDER BY COUNT(*) DESC) AS RN
  FROM tb_dengue_nova
  WHERE ANO IN (2023, 2024)
  GROUP BY ANO, ID_MUNICIPIO
) t
INNER JOIN tb_municipio_nova m
  ON t.ID_MUNICIPIO = m.ID_MUNICIPIO
WHERE t.RN = 1;

## MUNICIPIO QUE TEVE MAIS CASOS DE DENGUE

In [0]:
%sql
SELECT 
    m.MUNICIPIO,
    COUNT(*) AS TOTAL_CASOS
FROM tb_dengue_nova c
INNER JOIN tb_municipio_nova m ON c.ID_MUNICIPIO = m.ID_MUNICIPIO
GROUP BY m.MUNICIPIO
ORDER BY TOTAL_CASOS DESC
LIMIT 1

## QUANTIDADE DE CASOS COM FEBRE, VOMITO, NAUSEA, DOR MUSCULAR, DOR DE CABEÇA

In [0]:
%sql
SELECT
  SUM(CASE WHEN POSSUI_FEBRE = 'SIM' THEN 1 END) AS CASOS_FEBRE,
  SUM(CASE WHEN POSSUI_VOMITO = 'SIM' THEN 1 END) AS CASOS_VOMITO,
  SUM(CASE WHEN POSSUI_NAUSEA = 'SIM' THEN 1 END) AS CASOS_NAUSEA,
  SUM(CASE WHEN POSSUI_DOR_MUSCULAR = 'SIM' THEN 1 END) AS CASOS_DOR_MUSCULAR,
  SUM(CASE WHEN POSSUI_DOR_DE_CABECA = 'SIM' THEN 1 END) AS CASOS_DOR_DE_CABECA
FROM tb_dengue_nova

## SEXO QUE MAIS TEVE CASOS DE DENGUE

In [0]:
%sql
SELECT 
    c.SEXO AS SEXO,
    COUNT(*) AS TOTAL_CASOS
FROM tb_dengue_nova c
INNER JOIN tb_municipio_nova m 
    ON c.ID_MUNICIPIO = m.ID_MUNICIPIO
GROUP BY c.SEXO
ORDER BY TOTAL_CASOS DESC

In [0]:
df_gold = spark.sql("""
SELECT 
    c.ID_MUNICIPIO,
    m.MUNICIPIO,
    m.LATITUDE,
    m.LONGITUDE,
    m.ALTITUDE,
    c.DT_NOTIFICACAO,
    c.ANO,
    c.SEXO,
    c.POSSUI_FEBRE,
    c.POSSUI_NAUSEA,
    c.POSSUI_VOMITO,
    c.POSSUI_DOR_MUSCULAR,
    c.POSSUI_DOR_DE_CABECA,
    current_timestamp() AS DT_INGESTAO
FROM tb_dengue_nova c
INNER JOIN tb_municipio_nova m ON c.ID_MUNICIPIO = m.ID_MUNICIPIO""")

In [0]:
df_gold.write.mode("overwrite").parquet("/mnt/dados/gold/dengue_municipio_gold.parquet")

In [0]:
df_gold.createOrReplaceTempView("tb_dengue_gold")

In [0]:
display(spark.sql("SELECT * FROM tb_dengue_gold"))

In [0]:
display(dbutils.fs.ls("/mnt/dados/gold"))